In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

df = pd.read_csv('train.csv')
df['label'] = df['class'].apply(lambda x: 1 if x == 1 else 0)
df = df[['tweet', 'label']]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katarina.stanojkovic\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\katarina.stanojkovic\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\katarina.stanojkovic\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\katarina.stanojkovic\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
def preprocess_text(df, text_column):
    """
    Funkcija za predobradu teksta.
    
    Argumenti:
    - df: Pandas DataFrame koji sadrži kolonu sa tekstom.
    - text_column: Ime kolone koja sadrži tekstualne podatke.
    
    Vraća:
    - df: Pandas DataFrame sa novom kolonom 'clean_text' koja sadrži predobrađen tekst.
    """
    # Kopiramo originalni DataFrame da ne bismo menjali originalne podatke
    df = df.copy()
    
    # Uklanjanje URL-ova i email adresa
    df['clean_tweet'] = df[text_column].apply(lambda x: re.sub(r'http\S+|www.\S+|mailto:\S+', '', x))
    
    # Uklanjanje HTML tagova
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
    
    # Uklanjanje emotikona i specijalnih karaktera
    def remove_emojis(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emotikoni
                                   u"\U0001F300-\U0001F5FF"  # simobli i ikone
                                   u"\U0001F680-\U0001F6FF"  # transport i simobli
                                   u"\U0001F1E0-\U0001F1FF"  # zastave
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    df['clean_tweet'] = df['clean_tweet'].apply(remove_emojis)
    
    # Uklanjanje specijalnih karaktera i interpunkcije
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'[^A-Za-zšđčćžŠĐČĆŽ ]+', ' ', x))
    
    # Pretvaranje u mala slova
    df['clean_tweet'] = df['clean_tweet'].str.lower()
    
    # Uklanjanje dijakritika
    def remove_diacritics(text):
        text = unicodedata.normalize('NFKD', text)
        text = ''.join([c for c in text if not unicodedata.combining(c)])
        return text

    df['clean_tweet'] = df['clean_tweet'].apply(remove_diacritics)
    
    # Uklanjanje višestrukih razmaka
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('\s+', ' ', x).strip())
    
    # Tokenizacija
    df['tokens'] = df['clean_tweet'].apply(nltk.word_tokenize)
    
    # Uklanjanje stop-reči
    stop_words = set(stopwords.words('english'))  # Ako imaš stop-reči za srpski, zameni ovde
    df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Lematizacija
    lemmatizer = WordNetLemmatizer()
    df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    # Spajanje tokena nazad u string
    df['clean_tweet'] = df['tokens'].apply(lambda x: ' '.join(x))
    
    # Uklanjanje nepotrebnih kolona
    df = df.drop(columns=['tokens'])
    
    return df


In [3]:
data = preprocess_text(df, 'tweet')


C:\Users\katarina.stanojkovic\AppData\Local\Temp\ipykernel_11304\2996977020.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['clean_tweet'] = df['clean_tweet'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


In [4]:
from textattack.augmentation import EmbeddingAugmenter
from textattack.augmentation import Augmenter, BackTranslationAugmenter
from textattack.transformations import WordInsertionMaskedLM, WordSwapWordNet
from textattack.augmentation.recipes import CLAREAugmenter

c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
from textattack.augmentation import (
    EmbeddingAugmenter,
    CLAREAugmenter
)

import random

# Inicijalizacija TextAttack augmentera
embedding_augmenter = EmbeddingAugmenter()
clare_augmenter = CLAREAugmenter(model='distilroberta-base', tokenizer='distilroberta-base')

# Definisanje wrapper funkcija
def embedding_augment(text, **kwargs):
    augmented_texts = embedding_augmenter.augment(text)
    return random.choice(augmented_texts) if augmented_texts else text

def clare_augment(text, **kwargs):
    augmented_texts = clare_augmenter.augment(text)
    return random.choice(augmented_texts) if augmented_texts else text

# Lista TextAttack augmentera kao funkcija
textattack_methods = [
    clare_augment,
    embedding_augment
]


c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and ot

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
def augment_dataset_parallel(df, text_column, method, num_augmented_instances, max_workers=4, **kwargs):
    """
    Primeni izabranu metodu augmentacije na dataset koristeći paralelizaciju.
    
    Argumenti:
    - df: Originalni DataFrame.
    - text_column: Naziv kolone sa tekstom.
    - method: Funkcija metode augmentacije.
    - num_augmented_instances: Broj instanci koje treba generisati.
    - max_workers: Broj paralelnih radnika.
    - **kwargs: Dodatni argumenti za metodu augmentacije.
    
    Vraća:
    - DataFrame sa augmentiranim podacima.
    """
    augmented_texts = []
    indices = df.index.tolist()
    num_samples = len(indices)
    
    # Ako je broj instanci veći od broja dostupnih uzoraka, uzmi uzorke sa zamjenom
    replace = num_augmented_instances > num_samples
    sampled_indices = np.random.choice(indices, size=num_augmented_instances, replace=replace)
    
    def augment_text(idx):
        original_text = df.loc[idx, text_column]
        try:
            augmented_text = method(original_text, **kwargs)
            return augmented_text
        except Exception as e:
            print(f"Greška pri augmentaciji teksta na indeksu {idx}: {e}")
            return original_text  # Vraća originalni tekst u slučaju greške
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Koristimo tqdm za prikaz napretka
        futures = {executor.submit(augment_text, idx): idx for idx in sampled_indices}
        for future in tqdm(as_completed(futures), total=num_augmented_instances, desc=f'Augmenting with {method.__name__}'):
            augmented_texts.append(future.result())
    
    augmented_df = pd.DataFrame({text_column: augmented_texts})
    return augmented_df

In [8]:
negative_class_df = data[data['label'] == 0]

# Broj instanci po klasi
num_class_0 = len(negative_class_df)
num_class_1 = len(data[data['label'] == 1])

# Izračunajte razliku
class_difference = abs(num_class_0 - num_class_1)

# Broj metoda
num_methods = len(textattack_methods)

# Podelite negativnu klasu na delove po broju metoda
split_dataframes = np.array_split(negative_class_df, num_methods)

# Broj augmentisanih instanci po metodi
num_instances_per_method = class_difference // num_methods

augmented_dfs = []
for method, split_df in zip(textattack_methods, split_dataframes):
    if method==clare_augment:
        num_instances_per_method=2000
    else:
        num_instances_per_method=11000
    augmented_df = augment_dataset_parallel(
        split_df, 
        text_column='clean_tweet', 
        method=method, 
        num_augmented_instances=num_instances_per_method
    )
    augmented_df['label'] = 0
    augmented_dfs.append(augmented_df)

final_augmented_df = pd.concat(augmented_dfs, ignore_index=True)

# Kombinujte originalni i augmentisani dataset
final = pd.concat([data, final_augmented_df], ignore_index=True)

print("Originalni dataset:", data.shape)
print("Augmentisani dataset:", final_augmented_df.shape)
print("Konačni dataset:", final.shape)

final.to_csv('augmented_train.csv', index=False, encoding='utf-8')
print("Augmentirani dataset je sačuvan u 'augmented_train.csv'")

c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Augmenting with clare_augment:   0%|          | 0/2000 [00:00<?, ?it/s]

2024-11-15 19:08:40,007 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, VERB, PUNCT, ADP, DET, PROPN, PRON, ADJ, ADV, CCONJ, PART, NUM, AUX, INTJ, SYM, X, <START>, <STOP>
2024-11-15 19:08:40,109 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, VERB, PUNCT, ADP, DET, PROPN, PRON, ADJ, ADV, CCONJ, PART, NUM, AUX, INTJ, SYM, X, <START>, <STOP>



From c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



From c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_hub\resolver.py:369: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



From c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



From c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_hub\module_v2.py:126: The name tf.saved_model.load_v2 is deprecated. Please use tf.compat.v2.saved_model.load instead.

Augmenting with embedding_augment: 100%|██████████| 11000/11000 [14:15<00:00, 12.86it/s] 

Originalni dataset: (24783, 3)
Augmentisani dataset: (13000, 2)
Konačni dataset: (37783, 3)
Augmentirani dataset je sačuvan u 'augmented_train.csv'


In [3]:
augmented_df = pd.read_csv('augmented_train.csv')

print("Konačni augmentirani dataset:", augmented_df.shape)

Konačni augmentirani dataset: (37783, 3)


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Inicijalizacija GPT-2 modela i tokenizera
gpt_model_name = 'gpt2-medium'  # Možete koristiti 'gpt2', 'gpt2-large', itd.
gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
gpt_model.eval()
if torch.cuda.is_available():
    gpt_model.to('cuda')

def gpt_augment(text, max_length=50, num_return_sequences=1, temperature=0.7):
    """
    Generiše augmentisani tekst koristeći GPT-2 model.

    Argumenti:
    - text: Originalni tekst za augmentaciju.
    - max_length: Maksimalna dužina generisanog teksta.
    - num_return_sequences: Broj generisanih sekvenci.
    - temperature: Kontrola kreativnosti generisanog teksta.

    Vraća:
    - Generisani tekst.
    """
    input_ids = gpt_tokenizer.encode(text, return_tensors='pt')
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
    
    with torch.no_grad():
        outputs = gpt_model.generate(
            input_ids,
            max_length=len(input_ids[0]) + max_length,
            num_return_sequences=num_return_sequences,
            temperature=temperature,
            top_p=0.95,
            top_k=50,
            do_sample=True,
            eos_token_id=gpt_tokenizer.eos_token_id,
            pad_token_id=gpt_tokenizer.eos_token_id
        )
    
    generated_text = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Uklonite originalni tekst iz generisanog teksta
    augmented_text = generated_text[len(text):].strip()
    return augmented_text if augmented_text else text


c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\katarina.stanojkovic\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develo

In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Inicijalizacija T5 modela i tokenizera
t5_model_name = 't5-small'  # Možete koristiti 't5-base', 't5-large', itd.
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)
t5_model.eval()
if torch.cuda.is_available():
    t5_model.to('cuda')

def t5_paraphrase(text, max_length=50, num_return_sequences=1, num_beams=5):
    """
    Generiše parafrazirani tekst koristeći T5 model.

    Argumenti:
    - text: Originalni tekst za parafraziranje.
    - max_length: Maksimalna dužina generisanog teksta.
    - num_return_sequences: Broj generisanih sekvenci.
    - num_beams: Broj zrakova za beam search.

    Vraća:
    - Lista parafraziranih tekstova.
    """
    input_text = "paraphrase: " + text
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt', truncation=True)
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
    
    with torch.no_grad():
        outputs = t5_model.generate(
            input_ids,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            early_stopping=True
        )
    
    paraphrased_texts = [t5_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_texts


c:\Users\katarina.stanojkovic\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\katarina.stanojkovic\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.model

In [14]:

# Filtriranje negativne klase
negative_class_df = augmented_df[augmented_df['label'] == 0]

# Prikaz broja instanci u negativnoj klasi
print("Broj negativnih instanci:", negative_class_df.shape[0])

# Definisanje broja instanci za augmentaciju
# Na primer, augmentiramo 10.000 instanci
num_augmented_instances = 5000

# Ako je potrebno, odaberite nasumično podgrupu
selected_df = negative_class_df.sample(n=num_augmented_instances, random_state=42)

gpt_df = selected_df.sample(n=1000, random_state=42)
t5_df = selected_df.drop(gpt_df.index)


Broj negativnih instanci: 18593


In [15]:
from tqdm import tqdm

# Lista za skladištenje augmentisanih tekstova
gpt_augmented_texts = []

print("Primena GPT-based augmentacije...")
for text in tqdm(gpt_df['clean_tweet'], desc="GPT Augmentation"):
    augmented_text = gpt_augment(text)
    gpt_augmented_texts.append(augmented_text)

# Kreiranje DataFrame-a sa augmentiranim tekstovima
gpt_augmented_df = pd.DataFrame({
    'clean_tweet': gpt_augmented_texts,
    'label': 0
})


Primena GPT-based augmentacije...


GPT Augmentation: 100%|██████████| 1000/1000 [1:20:10<00:00,  4.81s/it]


In [16]:
# Lista za skladištenje augmentisanih tekstova
t5_augmented_texts = []

print("Primena T5-based parafraziranja...")
for text in tqdm(t5_df['clean_tweet'], desc="T5 Paraphrasing"):
    paraphrased_texts = t5_paraphrase(text)
    # Odabir jednog parafraziranog teksta
    augmented_text = paraphrased_texts[0] if paraphrased_texts else text
    t5_augmented_texts.append(augmented_text)

# Kreiranje DataFrame-a sa augmentiranim tekstovima
t5_augmented_df = pd.DataFrame({
    'clean_tweet': t5_augmented_texts,
    'label': 0
})


Primena T5-based parafraziranja...


T5 Paraphrasing: 100%|██████████| 4000/4000 [53:04<00:00,  1.26it/s]  


In [17]:
# Kombinovanje GPT-based i T5-based augmentacija
additional_augmented_df = pd.concat([gpt_augmented_df, t5_augmented_df], ignore_index=True)

# Kombinovanje sa postojećim augmentiranim dataset-om
final_augmented_df = pd.concat([augmented_df, additional_augmented_df], ignore_index=True)

# Prikaz veličine finalnog augmentiranog dataset-a
print("Originalni dataset:", augmented_df.shape)
print("Dodatni augmentirani podaci:", additional_augmented_df.shape)
print("Konačni augmentirani dataset:", final_augmented_df.shape)


Originalni dataset: (37783, 3)
Dodatni augmentirani podaci: (5000, 2)
Konačni augmentirani dataset: (42783, 3)


In [18]:
# Čuvanje konačnog augmentiranog dataset-a
final_augmented_df.to_csv('fully_augmented_train.csv', index=False, encoding='utf-8')
print("Konačni augmentirani dataset je sačuvan u 'fully_augmented_train.csv'")


Konačni augmentirani dataset je sačuvan u 'fully_augmented_train.csv'
